## Aufgabe 2 : Schätzung, Test, Prognose und Prognosefehler im multivariaten Modell ##

**Verwendeter Datensatz:** https://www.kaggle.com/datasets/mirichoi0218/insurance

**Ziel:** Vorhersage auf Basis verschiedener unabhängiger Variablen

**Allgemeines Vorgehen:**
1. Daten laden und aufbereiten
2. EDA (Vorstellung des Datensatzes) # jasmina
3. Modellspezifikation # jasmina
4. Modellschätzung # jasmina
6. F-Test # tina
7. Prognose (Fehler) und Prognoseintervalle # tina
8. Test der Annahmen bivariat + (Multikollinearität, Strukturbrüche, Homoskedastizität, Autokorrelation, Normalverteilung) # martin
9. Interpretation